SPAM MESSAGE CLASSIFIER <br>
Zuhayer Alvi <br>
CSC 44700 

For this project, we will create a spam message classifier using a public dataset from Kaggle (https://tinyurl.com/dsr5dww5)

The goal of this project is to classify messages as "spam" or "not spam" based on their content. <br>
By training a machine learning model on this labeled dataset, the model will learn to recognize patterns associated <br>
with spam messages, such as certain keywords, special characters, and the frequency of certain words. <br>


In [2]:
# Import all libraries needed
import string 
import numpy as np
import pandas as pd

import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Download nltk stop words package
# This package essentially has a collection of all commonly used words that can be removed from a text to focus on more important words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zuzu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Read and create a dataframe of our CSV file
df = pd.read_csv('spam_ham_dataset.csv')

# Take a look at the data frame
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


As we can see, this data frame has 4 columns and 5171 rows in total. <br> 

For this project, we only need to care about the 'text' and 'label_num' columns of this data frame. <br>
For all entries within the 'text' column, we will remove all linebreaks (\r and \n) and replace them with whitespace. <br>
This will allow us to ensure uniformality within the data frame. 

In [5]:
# Replace all \r and \n with whitespace
df['text'] = df['text'].apply(lambda x: x.replace('\r\n', ' '))

# Take a look at the new dataframe
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291 thi...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,3624,ham,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs this deal is to b...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft the transport vo...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,2933,ham,Subject: calpine daily gas nomination > > juli...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [6]:
# Now check the dataframe for any missing or nonexistant values 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [ ]:
# Begin with pre processing of all text: make everything lowercase, and stem words to shorten them
stemmer = PorterStemmer()
transformed_list = []